In [1]:
import numpy as np
import math
import pandas as pd
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression

# Spam

Se usarán diferentes clasificadores para clasificar spam.

In [2]:
## Calcula los verdaderos positivos, verdaderos negativos, falsos positivos y falsos negativos
# de un clasificador, dado un conjunto de prueba.
def evaluacion(clasificador, X_test, y_test):
    vp = 0
    vn = 0
    fp = 0
    fn = 0
    prediccion = clasificador.predict(X_test)
    for i in range(y_test.shape[0]):
        if y_test[i] == 1 and prediccion[i] == 1:
            vp += 1
        elif y_test[i] == 0 and prediccion[i] == 0:
            vn += 1
        elif y_test[i] == 1 and prediccion[i] == 0:
            fn += 1
        else: # y_test[i] == 0 and prediccion[i] == 1
            fp += 1
    return (vp, vn, fp, fn)

## Multinomial Naive Bayes

Lectura de datos y separación entre dos conjuntos. El primer conjunto se usará para entrenamiento y validación, mientras que el segundo será el de prueba.

In [3]:
datacsv = pd.read_csv('spam.csv', sep=' ')
datanp = datacsv.to_numpy()
Xinput = datanp[:, :-1]
yinput = datanp[:, -1]
X, X_pru, y, y_pru = train_test_split(Xinput, yinput, test_size=0.1, random_state=0)

Creación del objeto y pruebas.

In [4]:
nbc = MultinomialNB()
rskf = RepeatedStratifiedKFold(n_splits=5, n_repeats=10)
precision_nbc = 0
exhaustividad_nbc = 0
tasafp_nbc = 0
for train_index, test_index in rskf.split(X, y):
    X_train = X[train_index]
    y_train = y[train_index]
    X_test = X[test_index]
    y_test = y[test_index]
    nbc.fit(X_train, y_train)
    vp_nbc, vn_nbc, fp_nbc, fn_nbc = evaluacion(nbc, X_test, y_test)
    precision_nbc += vp_nbc / (vp_nbc + fp_nbc)
    exhaustividad_nbc += vp_nbc / (fn_nbc + vp_nbc)
    tasafp_nbc += fp_nbc / (vn_nbc + fp_nbc)

precision_nbc /= (5*10)
exhaustividad_nbc /= (5*10)
tasafp_nbc /= (5*10)
print('Precisión:', precision_nbc)
print('Exhaustividad:', exhaustividad_nbc)
print('Tasa de falsos positivos:', tasafp_nbc)

Precisión: 0.8903776102261868
Exhaustividad: 0.9432226322263226
Tasa de falsos positivos: 0.04766831705863475


## Regresión logística

Se entrena y se evalúa el modelo de regresión logística.

<b>Nota:</b> tuve que usar la regresión de sklearn porque en mi clase me enviaba un <i>OverflowError: math range error</i>.

In [5]:
rlog = LogisticRegression()
precision_rlog = 0
exhaustividad_rlog = 0
tasafp_rlog = 0
for train_index, test_index in rskf.split(X, y):
    X_train = X[train_index]
    y_train = y[train_index]
    X_test = X[test_index]
    y_test = y[test_index]
    rlog.fit(X_train, y_train)
    vp_rlog, vn_rlog, fp_rlog, fn_rlog = evaluacion(rlog, X_test, y_test)
    precision_rlog += vp_rlog / (vp_rlog + fp_rlog)
    exhaustividad_rlog += vp_rlog / (fn_rlog + vp_rlog)
    tasafp_rlog += fp_rlog / (vn_rlog + fp_rlog)

precision_rlog /= (5*10)
exhaustividad_rlog /= (5*10)
tasafp_rlog /= (5*10)
print('Precisión:', precision_rlog)
print('Exhaustividad:', exhaustividad_rlog)
print('Tasa de falsos positivos:', tasafp_rlog)

Precisión: 0.9316465106282255
Exhaustividad: 0.9740959409594094
Tasa de falsos positivos: 0.02937651858983175


Ahora se evalúa el modelo de regresión logística en el conjunto de prueba porque fue el que tuvo una precisión mayor.

In [6]:
vp_rlog2, vn_rlog2, fp_rlog2, fn_rlog2 = evaluacion(rlog, X_pru, y_pru)
precision_rlog2 = vp_rlog2 / (vp_rlog2 + fp_rlog2)
exhaustividad_rlog2 = vp_rlog2 / (fn_rlog2 + vp_rlog2)
tasafp_rlog2 = fp_rlog2 / (vn_rlog2 + fp_rlog2)
print('Precisión:', precision_rlog2)
print('Exhaustividad:', exhaustividad_rlog2)
print('Tasa de falsos positivos:', tasafp_rlog2)

Precisión: 0.96
Exhaustividad: 0.972972972972973
Tasa de falsos positivos: 0.016216216216216217
